 The command pip install requests and pip install pycoingecko are used to install Python packages.


In [55]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [56]:
pip install pycoingecko

Note: you may need to restart the kernel to use updated packages.



The code imports necessary Python modules:

pandas as pd: For data manipulation and analysis.
datetime as dt: For handling dates and times.
plotly.graph_objects as go and from plotly.offline import plot: For interactive data visualization, while from pycoingecko import CoinGeckoAPI imports CoinGeckoAPI for accessing cryptocurrency data

In [57]:
#import modules
import pandas as pd
import datetime as dt
import time as t
import plotly.graph_objects as go
from plotly.offline import plot
from pycoingecko import CoinGeckoAPI

In [58]:
#create a client
cg = CoinGeckoAPI()

#confirm connection
cg.ping()

{'gecko_says': '(V3) To the Moon!'}

In [59]:
#get a list of coins, sort df by id
coinList = cg.get_coins_list()
coinDataFrame = pd.DataFrame.from_dict(coinList).sort_values('id'
                                      ).reset_index(drop=True)



In [60]:
#btc/eth/dpx by id
#coinDataFrame[coinDataFrame['id'] == 'bitcoin']
#coinDataFrame[coinDataFrame['id'] == 'ethereum']
#coinDataFrame[coinDataFrame['id'] == 'dopex']
coins = ['bitcoin','ethereum','dopex']

#get list of suppored VS currencies
counterCurrencies = cg.get_supported_vs_currencies()
vsCurrencies = ['usd', 'eur', 'link']

#most simple price request - nested dictionary format
simplePriceRequest = cg.get_price(ids = coins, vs_currencies = 'usd')
print(simplePriceRequest)

{'bitcoin': {'usd': 62411}, 'dopex': {'usd': 54.73}, 'ethereum': {'usd': 3431.92}}


The script retrieves OHLC (Open, High, Low, Close) data for Bitcoin (BTC) from the CoinGecko API for a specified duration of 14 days.
It organizes the data into a pandas DataFrame, converting the timestamp to a human-readable date format and setting it as the index.
Finally, it prints the DataFrame, displaying the OHLC data in tabular format with the date as the index, suitable for analysis and visualization.

In [61]:
#get OHLC data for preset range 1/7/14/30/90/180/365/max
#candle body width by date range 
#1 - 2 days: 30 minutes
#3 - 30 days: 4 hours
#31 days and beyond: 4 days
ohlcData = cg.get_coin_ohlc_by_id(id = 'bitcoin', 
                                  vs_currency = 'usd', 
                                  days = '14')
#list to dataframe
ohlcDataFrame = pd.DataFrame(data = ohlcData,
                           columns = ['Date', 'Open', 'High' ,'Low', 'Close'])
#reformat date
ohlcDataFrame['Date'] = ohlcDataFrame['Date'].apply(
                        lambda x: dt.datetime.fromtimestamp(x/1000
                        ).strftime('%m-%d-%Y %H:%M:%S'))
#set index
ohlcDataFrame = ohlcDataFrame.set_index('Date')
print (ohlcDataFrame)



                        Open     High      Low    Close
Date                                                   
02-16-2024 14:00:00  51981.0  51981.0  51981.0  51981.0
02-16-2024 18:00:00  51755.0  52019.0  51755.0  52002.0
02-16-2024 22:00:00  52166.0  52166.0  51983.0  51983.0
02-17-2024 02:00:00  51988.0  52031.0  51930.0  51961.0
02-17-2024 06:00:00  51953.0  51953.0  51702.0  51702.0
...                      ...      ...      ...      ...
02-29-2024 22:00:00  61201.0  61509.0  60925.0  60925.0
03-01-2024 02:00:00  61162.0  61543.0  61162.0  61543.0
03-01-2024 06:00:00  61479.0  62197.0  61479.0  62059.0
03-01-2024 10:00:00  61907.0  62459.0  61907.0  61941.0
03-01-2024 14:00:00  61270.0  61907.0  61270.0  61907.0

[85 rows x 4 columns]


This script continuously fetches OHLC (Open, High, Low, Close) data for Bitcoin (BTC) from the CoinGecko API.
It converts the fetched data into a pandas DataFrame, organizing it into columns such as timestamp, open, high, low, and close.
The script prints the DataFrame, providing real-time updates on Bitcoin's OHLC data every second.

In [ ]:
import requests
import pandas as pd
import time

def fetch_ohlc_data(coin_id, vs_currency='usd', days=1):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/ohlc?vs_currency={vs_currency}&days={days}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
        return None

if __name__ == "__main__":
    while True:
        # Fetch OHLC data for Bitcoin (BTC) for the last 1 day
        ohlc_data = fetch_ohlc_data('bitcoin')

        if ohlc_data:
            print("OHLC Data:")
            # Convert OHLC data to a DataFrame
            df = pd.DataFrame(ohlc_data, columns=['timestamp', 'open', 'high', 'low', 'close'])
            print(df)
        else:
            print("Failed to fetch OHLC data.")

        # Wait for 1 second before making the next request
        time.sleep(1)


This script fetches OHLC (Open, High, Low, Close) data for Bitcoin (BTC) from the CoinGecko API at 5-second intervals for 1 hour.
It accumulates the fetched data into a list and formats it into a dictionary with timestamp, id, open, high, low, and close attributes.
Finally, it creates a pandas DataFrame from the accumulated data and prints it, presenting the OHLC data in a tabular format for analysis.

In [ ]:
import requests
import pandas as pd
import time

def fetch_ohlc_data(coin_id, vs_currency='usd', days=1):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/ohlc?vs_currency={vs_currency}&days={days}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
        return None

if __name__ == "__main__":
    # Initialize an empty list to accumulate OHLC data
    ohlc_accumulated_data = []

    # Fetch OHLC data for Bitcoin (BTC) for the last 1 day every 5 seconds, accumulate in the list
    for _ in range(12):  # Fetch for 1 hour (12 iterations, 5 seconds each)
        ohlc_data = fetch_ohlc_data('bitcoin')
        if ohlc_data:
            # Extracting required columns and appending to the accumulated data
            formatted_data = [{'timestamp': item[0], 'id': item[0], 'open': item[1], 'high': item[2], 'low': item[3], 'close': item[4]} for item in ohlc_data]
            ohlc_accumulated_data.extend(formatted_data)
            print("Batch data fetched and accumulated.")
        else:
            print("Failed to fetch batch OHLC data.")

        # Wait for 5 seconds before making the next request
        time.sleep(5)

    # Create a DataFrame from the accumulated data
    df = pd.DataFrame(ohlc_accumulated_data)

    # Display the DataFrame
    print("DataFrame Table:")
    print(df)


In [ ]:
pip install confluent-kafka

In [ ]:
pip install kafka-python


from kafka import KafkaProducer, KafkaConsumer: Imports the KafkaProducer and KafkaConsumer classes from the kafka library, allowing interaction with Apache Kafka message broker.
formatted_data_bytes = json.dumps(formatted_data).encode('utf-8'): Encodes the formatted_data dictionary into bytes after converting it to a JSON string.
producer = KafkaProducer(bootstrap_servers='localhost:19092'): Creates a KafkaProducer instance connected to the specified bootstrap servers, ready to send messages to the Kafka topic named 'topic'. Similarly, KafkaConsumer is initialized to consume messages from the same topic.

In [ ]:
from kafka import KafkaProducer, KafkaConsumer
import json 

formatted_data_bytes = json.dumps(formatted_data).encode('utf-8')

bootstrap_servers = 'localhost:19092'

producer = KafkaProducer(bootstrap_servers = bootstrap_servers) 

producer.send('topic', formatted_data_bytes)

consumer = KafkaConsumer(
  'topic',
  bootstrap_servers = bootstrap_servers  
)


import psycopg2: Imports the psycopg2 library, which is used for interacting with PostgreSQL databases in Python.
conn = psycopg2.connect(...): Establishes a connection (conn) to a PostgreSQL database using specified connection parameters like host, port, database name, username, and password.
This code connects to a PostgreSQL database named "data_challenge" running on localhost at port 5432 with the username "postgres" and password "postgres

In [ ]:

pip install psycopg2
import psycopg2

conn = psycopg2.connect(
   host="localhost",
   port="5432",
   database="data_challenge", 
   user="postgres",
   password="postgres"
)

cur = conn.cursor(): Creates a cursor object cur for executing SQL commands on a database connection conn.
for message in consumer:: Iterates over messages received from a Kafka topic.
cur.execute(sql, (timestamp,id,open, high,low,close)) inserts extracted data into a database table using SQL command sql, and conn.commit() commits the transaction to the database.


In [ ]:
cur = conn.cursor()

for message in consumer:
  # extract data
  timestamp = message.value['timestamp']
  id = message.value['id']
  open = message.value['open']
  high = message.value['high']
  low = message.value['low']
  close = message.value['close']

  sql = """INSERT INTO ticks2 (timestamp,id,open, high,low,close) 
           VALUES (timestamp,id,open, high,low,close)"""

  cur.execute(sql, (timestamp,id,open, high,low,close))

conn.commit()